 ## Display labeled tiles


1. small space between patches 
2. positive tiles marked with bright color

In [2]:
import cv2
import math
import os 
import shutil
import numpy as np

#label tiles
path_dir = r'\S10-16955 A7_201305011058.png'
path_dir_mask = r'\ECSE484_Fall2020\training_2020\CWRU_masks\S10-16955 A7_201305011058.png'
img_WSI = cv2.imread(path_dir)
image_YUV = cv2.cvtColor(img_WSI,cv2.COLOR_RGB2YUV)
img_mask = cv2.imread(path_dir_mask)
img_shape = img_WSI.shape
all_cropped_images = [];
all_cropped_masks = [];
positive_collection = []
negative_collection = []
non_tissue_collection = []
tile_size = (101, 101)
offset = (101, 101)
height  = int(math.ceil(img_shape[0]/(offset[1] * 1.0)))
width = int(math.ceil(img_shape[1]/(offset[0] * 1.0)))
for i in range(int(math.ceil(img_shape[0]/(offset[1] * 1.0)))):
    for j in range(int(math.ceil(img_shape[1]/(offset[0] * 1.0)))):
    #for each tile
    #step1. tile sampling
        cropped_img_WSI = img_WSI[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
        #all_cropped_images.append(cropped_img_WSI.astype('float'))
        cropped_img_mask = img_mask[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
        all_cropped_images.append(cropped_img_WSI)
        all_cropped_masks.append(cropped_img_mask)
        if cropped_img_WSI is not None:
            cropped_image_WSI_01 = cropped_img_WSI/255 
            cropped_image_WSI_img_01_mean = cropped_image_WSI_01.mean()
            cropped_image_WSI_img_01_std = cropped_image_WSI_01.std() 
            #step2.Remove non_tissue
            if cropped_image_WSI_img_01_mean >= 0.85 and cropped_image_WSI_img_01_std <= 0.1: 
                non_tissue_collection.append(str(i)+'_'+ str(j))
            else:
                a = np.mean(cropped_img_mask[:,:,0])/255 # binary mask 0：white; 1: black
                # step3: labeling
                if a >= 0.55:#60% white ； positive
                    positive_collection.append(str(i)+'_'+ str(j))
                else:
                    negative_collection.append(str(i) +'_' +str(j)) 
        else:
            non_tissue_collection.append(str(i)+'_' +str(j))

In [3]:
#check if negative tiles overlaps with non_tissue_collection tiles
import collections
negative_collection_multiset = collections.Counter(negative_collection)
non_tissue_collection_multiset = collections.Counter(non_tissue_collection)
overlap = list((negative_collection_multiset &non_tissue_collection_multiset).elements())

Display a WSI example with labeled tiles

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig_img, ax_img = plt.subplots(height,width, figsize=(300,250)) #figsize=(40,30) : width, height in inches.
bottom = 0.3; left=0.3
top=1.-bottom; right = 1.-left
plt.subplots_adjust(wspace = 0 ,hspace = 0, top=top, bottom=bottom, left=left, right=right )
for idx, img in enumerate(all_cropped_images):
    text = ax_img[int(idx/width), idx%width].text(20,20,int(str(int(idx/width))+str(int(idx%width)))  ,
                       ha="center", va="center", color="darkblue", fontsize=60)
    if  str(int(idx/width))+'_'+str(int(idx%width)) in  non_tissue_collection :
        text = ax_img[int(idx/width), idx%width].text(50,50, 'X',
                       ha="center", va="center", color="darkblue", fontsize=100)
        ax_img[int(idx/width), idx%width].axis('off')
    if  str(int(idx/width))+'_'+str(int(idx%width)) in positive_collection :#mark positivie tiles
        text = ax_img[int(idx/width), idx%width].text(50,50, 'P',
                       ha="center", va="center", color="darkblue", fontsize=100)
        #modify the style of spines
        ax_img[int(idx/width), idx%width].axis('on')
        ax_img[int(idx/width), idx%width].spines["top"].set_color("orangered") 
        ax_img[int(idx/width), idx%width].spines["bottom"].set_color("orangered") 
        ax_img[int(idx/width), idx%width].spines["left"].set_color("orangered") 
        ax_img[int(idx/width), idx%width].spines["right"].set_color("orangered")
        ax_img[int(idx/width), idx%width].spines["top"].set_linewidth(20) 
        ax_img[int(idx/width), idx%width].spines["bottom"].set_linewidth(20) 
        ax_img[int(idx/width), idx%width].spines["left"].set_linewidth(20) 
        ax_img[int(idx/width), idx%width].spines["right"].set_linewidth(20)
    if  str(int(idx/width))+'_'+str(int(idx%width)) in   negative_collection :#mark negative tiles
        text = ax_img[int(idx/width), idx%width].text(50,50, 'N',
                       ha="center", va="center", color="k", fontsize=100)
        ax_img[int(idx/width), idx%width].axis('on')
        ax_img[int(idx/width), idx%width].spines["top"].set_color("yellowgreen") 
        ax_img[int(idx/width), idx%width].spines["bottom"].set_color("yellowgreen") 
        ax_img[int(idx/width), idx%width].spines["left"].set_color("yellowgreen") 
        ax_img[int(idx/width), idx%width].spines["right"].set_color("yellowgreen")
        ax_img[int(idx/width), idx%width].spines["top"].set_linewidth(20) 
        ax_img[int(idx/width), idx%width].spines["bottom"].set_linewidth(20) 
        ax_img[int(idx/width), idx%width].spines["left"].set_linewidth(20) 
        ax_img[int(idx/width), idx%width].spines["right"].set_linewidth(20)

    ax_img[int(idx/width), idx%width].imshow(img)
    #ax_img[int(idx/width), idx%width].axis('off')
    
plt.savefig(os.path.join('\ECSE484_Fall2020', 'Display_labelled_tile.png'))
plt.show()

Display mask with labeled tiles

In [ ]:
import matplotlib.pyplot as plt
 
fig_img, ax_mask = plt.subplots(height,width, figsize=(300,250)) #figsize=(40,30) : width, height in inches.

for idx, img in enumerate(all_cropped_masks):
    text = ax_mask[int(idx/width), idx%width].text(50,50,int(str(int(idx/width))+str(int(idx%width)))  ,
                       ha="center", va="center", color="y", fontsize=200) # 50,50 : center
    ax_mask[int(idx/width), idx%width].imshow(img)
    ax_mask[int(idx/width), idx%width].axis('off')
    
plt.savefig(os.path.join('\ECSE484_Fall2020', 'Display_mask_tile.png'))  
plt.show()